In [5]:
# 이거 사용
import json
import mysql.connector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time
from datetime import datetime

# MySQL 연결 정보 설정
config = {
    'user': 'yangjinan',  # 사용자 이름
    'password': 'ALsk85200!!',  # 비밀번호
    'host': 'hanaex.mysql.database.azure.com',  # 호스트
    'database': 'my_database',  # 데이터베이스 이름
    'port': 3306  # 포트 번호
}

# state 변수 (EUR, USD 등)
state = "JPY"

# 오늘 날짜
today_date = datetime.today().strftime('%Y-%m-%d')

# 웹 드라이버 설정 (예: Chrome)
driver = webdriver.Chrome()

# 1. 페이지 로드
url = "https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_07i.do#//HanaBank"
driver.get(url)

# 2. 페이지 로드 대기
wait = WebDriverWait(driver, 10)

# 3. 로딩 레이어가 사라질 때까지 대기
try:
    loading_layer = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#OPB_loadingLayerID_generatedByJSOPB_modalMaskID_generatedByJS")))
    wait.until(EC.invisibility_of_element(loading_layer))
except TimeoutException:
    print("Loading layer did not disappear, continuing...")

# 6. 통화 선택 (EUR:유로(유럽연합))
currency_select = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#curCd")))
select = Select(currency_select)
select.select_by_value(state)

# 7. "조회" 버튼 클릭
query_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#HANA_CONTENTS_DIV > div.btnBoxCenter > a")))
query_button.click()

# 8. 조회 후 5초 대기
time.sleep(5)

# 9. 데이터 추출
data_list = []
rows = driver.find_elements(By.CSS_SELECTOR, "#searchContentDiv > div.printdiv > table > tbody > tr")

for row in rows:
    columns = row.find_elements(By.TAG_NAME, "td")
    data = [column.text for column in columns]
    
    # 필요한 값을 float로 변환 (첫 번째 값은 제외)
    for i in range(2, len(data)):
        try:
            data[i] = float(data[i].replace(',', ''))  # 쉼표 제거 후 float 변환
        except ValueError:
            print(f"변환 실패: {data[i]}")
    
    # 데이터 리스트에 추가
    data_list.append(data)
    print("Row:", data)

# 웹 드라이버 종료
driver.quit()

# 10. 데이터베이스에 데이터 저장
try:
    # MySQL 데이터베이스 연결
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    
    # SQL INSERT 문 작성
    query = """
    INSERT INTO t_ex_table 
    (state, period, time, cash_buy, cash_sell, remit_send, remit_receive, 
    foreign_currency_check_sell, exchange_rate, compared_to_previous, exchange_fee_rate, usd_conversion_rate, date) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    # 데이터베이스에 저장할 값 준비
    for data in data_list:
        values = (
            state,         # state 변수 저장
            int(data[0]),  # period
            data[1],       # time
            data[2],       # cash_buy
            data[3],       # cash_sell
            data[4],       # remit_send
            data[5],       # remit_receive
            data[6],       # foreign_currency_check_sell
            data[7],       # exchange_rate
            data[8],       # compared_to_previous
            data[9],       # exchange_fee_rate
            data[10],      # usd_conversion_rate
            today_date     # 오늘 날짜 저장
        )
        
        # 데이터 삽입
        cursor.execute(query, values)
    
    # 트랜잭션 커밋
    conn.commit()

except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    # 커넥션 닫기
    cursor.close()
    conn.close()

print("데이터 저장 완료!")

Row: ['170', '11:41:52', 920.11, 888.47, 913.15, 895.43, 894.92, 904.29, 0.2, 2.315, 0.6704]
Row: ['169', '11:40:52', 920.31, 888.67, 913.35, 895.63, 895.12, 904.49, 0.07, 2.315, 0.6705]
Row: ['168', '11:40:06', 920.24, 888.6, 913.28, 895.56, 895.05, 904.42, 0.14, 2.315, 0.6705]
Row: ['167', '11:39:37', 920.1, 888.46, 913.14, 895.42, 894.91, 904.28, 0.12, 2.315, 0.6705]
Row: ['166', '11:34:31', 919.98, 888.34, 913.02, 895.3, 894.79, 904.16, 0.13, 2.315, 0.6704]
Row: ['165', '11:32:32', 920.11, 888.47, 913.15, 895.43, 894.92, 904.29, 0.08, 2.315, 0.6704]
Row: ['164', '11:30:07', 920.19, 888.55, 913.23, 895.51, 895.0, 904.37, 0.19, 2.315, 0.6704]
Row: ['163', '11:29:07', 920.38, 888.74, 913.42, 895.7, 895.19, 904.56, 0.12, 2.315, 0.6704]
Row: ['162', '11:24:02', 920.51, 888.85, 913.54, 895.82, 895.31, 904.68, 0.02, 2.315, 0.6705]
Row: ['161', '11:19:01', 920.53, 888.87, 913.56, 895.84, 895.33, 904.7, 0.07, 2.315, 0.6704]
Row: ['160', '11:17:47', 920.6, 888.94, 913.63, 895.91, 895.4, 904.

Row: ['82', '10:00:36', 920.55, 888.89, 913.58, 895.86, 895.35, 904.72, 0.44, 2.315, 0.6703]
Row: ['81', '09:58:37', 920.1, 888.46, 913.14, 895.42, 894.91, 904.28, 0.13, 2.315, 0.6701]
Row: ['80', '09:57:51', 920.23, 888.59, 913.27, 895.55, 895.04, 904.41, 0.13, 2.315, 0.6701]
Row: ['79', '09:57:32', 920.1, 888.46, 913.14, 895.42, 894.91, 904.28, 0.32, 2.315, 0.6701]
Row: ['78', '09:56:38', 919.77, 888.15, 912.81, 895.11, 894.6, 903.96, 0.13, 2.315, 0.6699]
Row: ['77', '09:56:32', 919.64, 888.02, 912.68, 894.98, 894.47, 903.83, 0.14, 2.315, 0.67]
Row: ['76', '09:55:11', 919.5, 887.88, 912.54, 894.84, 894.33, 903.69, 0.25, 2.315, 0.6699]
Row: ['75', '09:54:26', 919.25, 887.63, 912.29, 894.59, 894.08, 903.44, 0.37, 2.315, 0.6699]
Row: ['74', '09:53:51', 919.62, 888.0, 912.66, 894.96, 894.45, 903.81, 0.01, 2.315, 0.6701]
Row: ['73', '09:52:32', 919.61, 887.99, 912.65, 894.95, 894.44, 903.8, 0.14, 2.315, 0.6702]
Row: ['72', '09:52:17', 919.75, 888.13, 912.79, 895.09, 894.58, 903.94, 0.23, 

In [7]:
# 실시간 크롤링
import json
import mysql.connector
from mysql.connector import Error
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException, ElementClickInterceptedException,
    NoSuchElementException, StaleElementReferenceException
)
import time
from datetime import datetime

# MySQL 연결 정보 설정
config = {
    'user': 'yangjinan',  # 사용자 이름
    'password': 'ALsk85200!!',  # 비밀번호
    'host': 'hanaex.mysql.database.azure.com',  # 호스트
    'database': 'my_database',  # 데이터베이스 이름
    'port': 3306  # 포트 번호
}

# state 변수 (EUR, USD 등)
state = "USD"

# 오늘 날짜
today_date = datetime.today().strftime('%Y-%m-%d')

# MySQL 데이터베이스 연결
try:
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
except mysql.connector.Error as err:
    print(f"Database connection error: {err}")
    exit(1)  # 연결 실패 시 프로그램 종료

# 웹 드라이버 설정 (예: Chrome)
driver = webdriver.Chrome()

# URL 설정
url = "https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_07i.do#//HanaBank"

try:
    while True:
        # 1. 페이지 로드
        driver.get(url)

        # 2. 페이지 로드 대기
        wait = WebDriverWait(driver, 10)

        # 3. 로딩 레이어가 사라질 때까지 대기
        try:
            loading_layer = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#OPB_loadingLayerID_generatedByJSOPB_modalMaskID_generatedByJS")))
            wait.until(EC.invisibility_of_element(loading_layer))
        except TimeoutException:
            print("Loading layer did not disappear, continuing...")
        except NoSuchElementException:
            pass  # 로딩 레이어가 없으면 통과

        # 6. 통화 선택 (예: EUR)
        try:
            currency_select = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#curCd")))
            select = Select(currency_select)
            select.select_by_value(state)
        except Exception as e:
            print(f"Error selecting currency: {e}")
            continue  # 다음 루프로 이동

        # 7. "조회" 버튼 클릭
        try:
            query_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#HANA_CONTENTS_DIV > div.btnBoxCenter > a")))
            query_button.click()
        except Exception as e:
            print(f"Error clicking query button: {e}")
            continue

        # 8. 조회 후 5초 대기
        time.sleep(5)

        # 9. 데이터 추출 (첫 번째 행만)
        try:
            first_row = driver.find_element(By.CSS_SELECTOR, "#searchContentDiv > div.printdiv > table > tbody > tr")
            columns = first_row.find_elements(By.TAG_NAME, "td")
            data = [column.text for column in columns]

            # 필요한 값을 float로 변환 (첫 번째 두 개의 값은 제외)
            for i in range(2, len(data)):
                try:
                    data[i] = float(data[i].replace(',', ''))  # 쉼표 제거 후 float 변환
                except ValueError:
                    print(f"변환 실패: {data[i]}")

            print("추출된 데이터:", data)
        except Exception as e:
            print(f"Error extracting data: {e}")
            continue

        # 10. 데이터베이스에서 기존 period 조회 및 비교
        try:
            # 동일한 state와 date에 대한 최대 period 조회
            cursor.execute("""
                SELECT MAX(period) FROM t_ex_table WHERE state = %s AND date = %s
            """, (state, today_date))
            result = cursor.fetchone()
            stored_period = result[0] if result[0] is not None else 0
        except mysql.connector.Error as err:
            print(f"Database query error: {err}")
            continue

        # period 비교 및 데이터 삽입
        try:
            scraped_period = int(data[0])
        except ValueError:
            print(f"Invalid period value: {data[0]}")
            continue

        if scraped_period > stored_period:
            # 데이터베이스에 데이터 저장
            try:
                query = """
                INSERT INTO t_ex_table 
                (state, period, time, cash_buy, cash_sell, remit_send, remit_receive, 
                foreign_currency_check_sell, exchange_rate, compared_to_previous, exchange_fee_rate, usd_conversion_rate, date) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                values = (
                    state,         # state 변수 저장
                    scraped_period,  # period
                    data[1],       # time
                    data[2],       # cash_buy
                    data[3],       # cash_sell
                    data[4],       # remit_send
                    data[5],       # remit_receive
                    data[6],       # foreign_currency_check_sell
                    data[7],       # exchange_rate
                    data[8],       # compared_to_previous
                    data[9],       # exchange_fee_rate
                    data[10],      # usd_conversion_rate
                    today_date     # 오늘 날짜 저장
                )
                cursor.execute(query, values)
                conn.commit()
                print("데이터 저장 완료!")
            except mysql.connector.Error as err:
                print(f"Database insertion error: {err}")
        else:
            print("새로 추출한 period가 저장된 period보다 크지 않습니다. 데이터 저장을 생략합니다.")

        # 1분 대기
        time.sleep(5)

except KeyboardInterrupt:
    print("프로세스가 사용자에 의해 중단되었습니다.")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

finally:
    # 리소스 정리
    driver.quit()
    cursor.close()
    conn.close()
    print("리소스를 정상적으로 종료하였습니다.")

추출된 데이터: ['204', '12:44:52', 1373.11, 1325.89, 1362.7, 1336.3, 1333.8, 1349.5, 0.2, 6.67299, 1.0]
데이터 저장 완료!
Loading layer did not disappear, continuing...
추출된 데이터: ['204', '12:44:52', 1373.11, 1325.89, 1362.7, 1336.3, 1333.8, 1349.5, 0.2, 6.67299, 1.0]
새로 추출한 period가 저장된 period보다 크지 않습니다. 데이터 저장을 생략합니다.
Loading layer did not disappear, continuing...
추출된 데이터: ['204', '12:44:52', 1373.11, 1325.89, 1362.7, 1336.3, 1333.8, 1349.5, 0.2, 6.67299, 1.0]
새로 추출한 period가 저장된 period보다 크지 않습니다. 데이터 저장을 생략합니다.
Loading layer did not disappear, continuing...
추출된 데이터: ['205', '12:49:57', 1373.11, 1325.89, 1362.7, 1336.3, 1333.8, 1349.5, 0.0, 6.67299, 1.0]
데이터 저장 완료!
Loading layer did not disappear, continuing...
추출된 데이터: ['205', '12:49:57', 1373.11, 1325.89, 1362.7, 1336.3, 1333.8, 1349.5, 0.0, 6.67299, 1.0]
새로 추출한 period가 저장된 period보다 크지 않습니다. 데이터 저장을 생략합니다.
프로세스가 사용자에 의해 중단되었습니다.
리소스를 정상적으로 종료하였습니다.
